# EE 578B Convex Optimization Project
## Kyle Hadley

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.lines as mlines
import cvxpy as cp
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

In [2]:
import warnings
warnings.simplefilter('ignore')

## Introduction

Working within the aviation industry, one of the most interesting convex optimization problems that airlines face involves deciding which routes should be offered to their customers. There are many different factors that are involved in choosing the routes: passenger demand, cost of operating route, aircraft available to operate the route, etc.

Based on these factors, there have been predominately two approaches to planning airline's routes: (1) the hub-and-spoke model or (2) point-to-point.

![point_hub_network](point_hub_network.png)

The hub-and-spoke model mimics the spokes of a wheel spreading out from a single; typically airlines has a few select hubs (depending on an airline's size) that all flights fly through. An example of this is Alaska Airlines hub based out of the Seattle-Tacoma airport. The majority of the flights operated by Alaska Air flows through this airport. In this model, the airline chooses to connect as many cities to a single hub such that customers can access a larger network of cities, but are likely to require at least 1 or 2 layover flights. This enables an airline to operate larger aircraft between major hubs (e.g. Los Angeles and New York) and then utilize smaller, regional aircraft for any small flights from the hub to smaller cities (e.g. Log Angeles to San Diego).

The point-to-point model instead attempts to connect as many cities directly rather than through a single hub. The point-to-point model emphasizes flying between two cities directly, regardless of size. An example of this approach is RyanAir which operates many low-cost flights through Europe continent. RyanAir attempts to connect as many cities, regardless of the size of cities to provide easy to access flights. Airlines can often more easily ensure that their routes are meeting the customer demand for a specific pair of cities as they don't need to be as concerned with ensuring "connecting" flights are supported.

There are certainly disadvantages to both solutions. A hub-and-spoke model often leads to higher facility costs and requires passengers to often make connecting flights to reach their final destination (ultimately leading to longer travel times). A point-to-point model also requires significantly more aircraft in a given fleet to support the same number of routes; sometimes this may mean that cities cannot be supported within a point-to-point model if an airline does not have adequate aircraft within their fleet.

Through my project, I hope to provide the foundation of a tool that could allow airlines to more quickly plan their routes based on customer demand. The model will compare multiple networks connected cities - following the hub-and-spoke model, point-to-point, and a combination of the two - to identify the optimal routes to maximize customer reward.

To help develop the preliminary aspects of the model, I'll be imagining that my model is to be used for an upstart airline - *Concave Airlines* - to help them identify how they should offer routes between various cities to support customer demands. *Concave Airlines* is based out of Las Vegas, Nevada and is focused on mainly offering routes along the west coast and mountain states of the US, but also is interested in potentially supporting other major cities across the US and in North America.

The following cities are currently of interest to be serviced by the airline:
- Las Vegas, NV
- Reno, NV
- Salt Lake City, UT
- Boise, ID
- Seattle, WA
- Los Angeles, CA
- San Francisco, CA

![graph of cities](City_Connections.png)

# Problem Setup

We can attempt to model our routes similar to the approach taken when modeling shortest path problems with edges formulated. Thus, we can think of the model as minimizing the cost of mass flow such that:

$$\min\limits_x c^Tx$$
$$\text{s.t.} \; Ex = b, x \geq 0$$

In order to leverage this general linear program, we must first start with a graph in order to define our matrices $c$, $E$, and $b$. To generate these matrices, we must evaluate the graph as it relates to servicing these cities.

The graph is dependent on how we initially setup our routes; thus, we must setup a few potential graphs and compare the results of each graph after minimizing the linear program. In each graph our nodes represent the cities we want to service and the edges represent flights that we will offer to customers to meet their needs.

Our first graph will utilize the hub-and-spoke model as discussed above; the graph is illustrated below:

![hub_and_spoke model](Hub_and_spoke_model.png)

From this graph, we can see that we have 7 nodes and 12 total edges. Thus, we can see that our $E \in \mathbb{R}^{7x12}$ and $b \in \mathbb{R}^{7}$.

Once of the initial issues that can be seen with the initial linear program is that the initial program assumes that we are only moving mass (i.e. our customers) from an initial node to a final node. However, we are hoping to service more than two nodes in our graph - we want to provide flights between multiple cities. In order to account for this, we can add additional constraints and $x's$. For example, if we were offering two routes we could re-write the problem as,

$$\min\limits_{x_1, x_2} c^T(x_1+x_2)$$
$$\text{s.t.} \; Ex_1 = b_1, Ex_2=b_2, x_i \geq 0 \forall i$$

We can add constraints and $x's$ dependent on the number of cities we want to connect. We'll discuss the number of constraints and $x's$ we add depending on the graph we're evaluating in the next section.

Another issue is that the initial problem we outlined doesn't account for larger mass flow (i.e. number of customers) between two nodes. All of our values of $b$ are currently binary vectors with a single $1$ and $-1$ indicating the initial and final node. In order to account for greater mass flow through a specific point, we can add a mass $m_i$ for each given constraint such that $Ex_i = m_i b_i$ where $m_i$ is a constant representing the number of customers flowing between nodes. For the purposes of our modeling, we will assume that the mass is a function of the population size of a given city (such as $0.05%$). It seems reasonable to expect that larger cities will have a greater mass flow leaving the city going to another city.

# References

- https://aeronauticsonline.com/the-airline-economics-of-the-bicycle-wheel-point-to-point-vs-hub-and-spoke-flying/
- https://transportgeography.org/contents/chapter2/geography-of-transportation-networks/point-to-point-versus-hub-and-spoke-network/#:~:text=A%20point%2Dto%2Dpoint%20network,intermediary%20location%20called%20a%20hub
- https://simpleflying.com/how-do-airlines-plan-routes/